In [3]:
#Creacion Variables en los CSV

import os
import pandas as pd
import numpy as np

# 📁 Carpeta de entrada y salida
CARPETA_ENTRADA = './Processed Data1/'
CARPETA_SALIDA = './Processed Data2/'
os.makedirs(CARPETA_SALIDA, exist_ok=True)

# 🔧 Función para calcular indicadores y target refinado
def refinar_dataset(df):
    # --- Features técnicas existentes ---
    df['sma20'] = df['close'].rolling(window=20).mean()

    delta = df['close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=20).mean()
    avg_loss = pd.Series(loss).rolling(window=20).mean()
    rs = avg_gain / (avg_loss + 1e-9)
    df['rsi20'] = 100 - (100 / (1 + rs))

    stddev = df['close'].rolling(window=20).std()
    df['bbhigh'] = df['sma20'] + 2 * stddev
    df['bblow'] = df['sma20'] - 2 * stddev

    # --- Nuevas features cuantitativas ---
    df['return_1h'] = df['close'].pct_change(periods=1)
    df['return_1d'] = df['close'].pct_change(periods=24)
    df['volatility_10'] = df['close'].rolling(window=10).std()
    df['volume_change'] = df['volume'].pct_change(periods=1)

    # --- Variables binarias auxiliares ---
    df['is_above_sma'] = (df['close'] > df['sma20']).astype(int)
    df['rsi_overbought'] = (df['rsi20'] > 70).astype(int)
    df['rsi_oversold'] = (df['rsi20'] < 30).astype(int)
    df['close_near_bbhigh'] = (df['close'] > df['bbhigh'] * 0.95).astype(int)

    # --- Target refinado: retorno futuro > 0.3% ---
    future_return = df['close'].shift(-1) / df['close'] - 1
    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 🧹 Eliminar valores nulos y periodo de pandemia
    df.dropna(inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df[~(df['timestamp'] <= '2022-07-31')]

    return df

# 🔁 Procesar todos los archivos
def refinar_todos_los_csv(carpeta_entrada, carpeta_salida):
    archivos = [f for f in os.listdir(carpeta_entrada) if f.endswith('.csv')]

    for archivo in archivos:
        ruta_entrada = os.path.join(carpeta_entrada, archivo)
        ruta_salida = os.path.join(carpeta_salida, archivo)

        df = pd.read_csv(ruta_entrada)
        df = refinar_dataset(df)
        df.to_csv(ruta_salida, index=False)

        print(f"✅ Refinado y guardado: {ruta_salida}")

# 🏁 Ejecutar
refinar_todos_los_csv(CARPETA_ENTRADA, CARPETA_SALIDA)


✅ Refinado y guardado: ./Processed Data2/AAPL.csv
✅ Refinado y guardado: ./Processed Data2/AMZN.csv
✅ Refinado y guardado: ./Processed Data2/GOOGL.csv
✅ Refinado y guardado: ./Processed Data2/META.csv
✅ Refinado y guardado: ./Processed Data2/MSFT.csv
✅ Refinado y guardado: ./Processed Data2/NVDA.csv
✅ Refinado y guardado: ./Processed Data2/TSLA.csv
